In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

## Load in all data from three sources:
### County government spending, Median HH Income, and Mortality

In [106]:
govt_spending = pd.read_csv("WellBeingData/govt_spending_data.csv")
median_HHincome = pd.read_excel("WellBeingData/MHH_ALL_COMBINED.xlsx", na_values=['.'])
mortality = pd.read_csv('WellBeingData/mortality.csv')

# print(len(govt_spending), len(median_HHincome), len(mortality))

# Prepare Govt Spending Data
govt_spending = govt_spending.dropna(subset=['State_Code', 'County'])
govt_spending['FIPS'] = govt_spending['State_Code'].astype(int).astype(str).str.zfill(2) + govt_spending['County'].astype(int).astype(str).str.zfill(3)
govt_spending.set_index(govt_spending['FIPS'].astype(str) + '_' + govt_spending['Year4'].astype(str), inplace=True)
govt_spending = govt_spending[
        ['FIPS', 'Year4', 'Name', 'Population'] +
        ['Total Revenue (non per capita)', 'Total_Educ_Total_Exp', 'Total_Construction', 'Public_Welf_Current_Exp']
    ]

# Prepare Median HH Income data
MHH_years = [col.split(' ')[3] for col in median_HHincome.columns if 'Median Household Income' in col]
median_HHincome['FIPS Code'] = median_HHincome['FIPS Code'].astype(str).str.zfill(5)
rows = []
for i, row in median_HHincome.iterrows():
    for y in MHH_years:
        r = [row['FIPS Code'], row['Postal Code'], row['Name'], y, float(row[f'Median Household Income {y}'])]
        rows.append(r)
MHH = pd.DataFrame(rows)
MHH.columns = ['FIPS', 'Postal Code', 'Name', 'Year', 'Median Household Income']
MHH.set_index(MHH['FIPS'].astype(str) + '_' + MHH['Year'].astype(str), inplace=True)


# Prepare Mortality data
mort_years = range(1981, 2021)
mortality = mortality.dropna(subset=['FIPS'])
mortality['FIPS'] = mortality['FIPS'].astype(int).astype(str).str.zfill(5)
rows = []
for i, row in mortality.iterrows():
    for y in mort_years:
        r = [row['FIPS'], y, row[f'Mortality Rate, {y}*']]
        rows.append(r)
mortality = pd.DataFrame(rows)
mortality.columns = ['FIPS', 'Year', 'Mortality Rate']
mortality.set_index(mortality['FIPS'].astype(str) + '_' + mortality['Year'].astype(str), inplace=True)


# Join all the data

## Inner join on FIPS_Year

r1 = pd.merge(govt_spending, MHH, left_index=True, right_index=True)
r2 = pd.merge(r1, mortality, left_index=True, right_index=True)

final = r2[[
    'FIPS', 'Year_x', 'Postal Code', 'Name_y', 'Population',
    'Total Revenue (non per capita)', 'Median Household Income', 'Mortality Rate',
    'Total_Educ_Total_Exp', 'Total_Construction', 'Public_Welf_Current_Exp'
]]

final.columns = [
    'FIPS', 'Year', 'State Code', 'County Name', 'Population',
    'Total_Revenue', 'Median Household Income', 'Mortality Rate',
    'Total_Educ_Total_Exp', 'Total_Construction', 'Public_Welf_Current_Exp'
]

print(final.head())
print(len(final))

             FIPS  Year State Code     County Name  Population  Total_Revenue  \
01001_1997  01001  1997         AL  Autauga County       40061           6951   
01001_2000  01001  2000         AL  Autauga County       43671           8302   
01001_2002  01001  2002         AL  Autauga County       43671          10579   
01001_2004  01001  2004         AL  Autauga County       45604          11386   
01001_2005  01001  2005         AL  Autauga County       48612          11843   

            Median Household Income  Mortality Rate  Total_Educ_Total_Exp  \
01001_1997                  36803.0         428.212                    28   
01001_2000                  45744.0         413.920                    28   
01001_2002                  45872.0         395.884                   250   
01001_2004                  45379.0         377.848                   275   
01001_2005                  45019.0         368.830                   275   

            Total_Construction  Public_Welf_Curren

### Check the completeness of our inner joined data set.

In [107]:
X_cols = ['Total_Educ_Total_Exp', 'Total_Construction', 'Public_Welf_Current_Exp']
Y_cols = ['Total_Revenue', 'Median Household Income', 'Mortality Rate']

for col in X_cols + Y_cols:
    print(f'{col} \t\t\t\t {(final[col] > 0).sum() / len(final)}')

Total_Educ_Total_Exp 				 0.3538105413105413
Total_Construction 				 0.7107668566001899
Public_Welf_Current_Exp 				 0.7914292497625831
Total_Revenue 				 0.9995251661918328
Median Household Income 				 0.9996438746438746
Mortality Rate 				 1.0


In [108]:
# Drop any nans

final = final.dropna(how='any')
print(len(final))

15442


In [109]:
# Convert some of them to per capita, convert from thousands of dollars to dollars.

for col in ['Total_Revenue', 'Total_Educ_Total_Exp', 'Total_Construction', 'Public_Welf_Current_Exp']:
    final[col] /= final['Population'] / 1e3

## Run regressions

In [110]:
#define predictor and response variables
y = final['Total_Revenue'] 
x = final[X_cols]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model1 = sm.OLS(y, x).fit()

#view model summary
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:          Total_Revenue   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     1206.
Date:                Fri, 26 May 2023   Prob (F-statistic):               0.00
Time:                        18:31:49   Log-Likelihood:            -1.3427e+05
No. Observations:               15442   AIC:                         2.685e+05
Df Residuals:                   15438   BIC:                         2.686e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [111]:
#define predictor and response variables
y = final['Median Household Income'] 
x = final[X_cols]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model2 = sm.OLS(y, x).fit()

#view model summary
print(model2.summary())

                               OLS Regression Results                              
Dep. Variable:     Median Household Income   R-squared:                       0.030
Model:                                 OLS   Adj. R-squared:                  0.030
Method:                      Least Squares   F-statistic:                     158.4
Date:                     Fri, 26 May 2023   Prob (F-statistic):          4.29e-101
Time:                             18:31:52   Log-Likelihood:            -1.6874e+05
No. Observations:                    15442   AIC:                         3.375e+05
Df Residuals:                        15438   BIC:                         3.375e+05
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [112]:
#define predictor and response variables
y = final['Mortality Rate'] 
x = final[X_cols]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model3 = sm.OLS(y, x).fit()

#view model summary
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:         Mortality Rate   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     189.1
Date:                Fri, 26 May 2023   Prob (F-statistic):          1.98e-120
Time:                        18:31:54   Log-Likelihood:                -88323.
No. Observations:               15442   AIC:                         1.767e+05
Df Residuals:                   15438   BIC:                         1.767e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

## Data results in Graphs and Tables

In [113]:
# Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
results_as_html = model.summary().tables[1].as_html()
pd.read_html(results_as_html, header=0, index_col=0)[0]

,coef,std err,t,P>|t|,[0.025,0.975]
const,309.0781,0.721,428.686,0.0,307.665,310.491
Total_Educ_Total_Exp,0.0243,0.001,17.634,0.0,0.022,0.027
Total_Construction,-0.0455,0.003,-15.149,0.0,-0.051,-0.040
Public_Welf_Current_Exp,-0.0490,0.004,-10.925,0.0,-0.058,-0.040


In [114]:
stargazer = Stargazer([model1, model2, model3])

In [115]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
\\[-1.8ex] & (1) & (2) & (3) \\
\hline \\[-1.8ex]
 Public_Welf_Current_Exp & 1.898$^{***}$ & 15.882$^{***}$ & -0.049$^{***}$ \\
  & (0.088) & (0.819) & (0.004) \\
 Total_Construction & 2.020$^{***}$ & 3.810$^{***}$ & -0.045$^{***}$ \\
  & (0.059) & (0.548) & (0.003) \\
 Total_Educ_Total_Exp & 0.911$^{***}$ & 0.546$^{**}$ & 0.024$^{***}$ \\
  & (0.027) & (0.252) & (0.001) \\
 const & 841.221$^{***}$ & 41657.846$^{***}$ & 309.078$^{***}$ \\
  & (14.127) & (131.680) & (0.721) \\
\hline \\[-1.8ex]
 Observations & 15,442 & 15,442 & 15,442 \\
 $R^2$ & 0.190 & 0.030 & 0.035 \\
 Adjusted $R^2$ & 0.190 & 0.030 & 0.035 \\
 Residual Std. Error & 1445.249(df = 15438) & 13471.786(df = 15438) & 73.762(df = 15438)  \\
 F Statistic & 1205.709$^{***}$ (df = 3.0; 15438.0) & 158.353$^{***}$ (df = 3.0; 15438.0) & 189.095$^{***}$ (df = 3.0; 15438.0) \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \mult

In [116]:
final.to_excel("WellBeingData/final_tabulated_data.xlsx")

## Run the same regressions, except for each year

In [118]:
models = []

for year in sorted(list(final['Year'].unique())):
    idx = final[final['Year'] == year]

    #define predictor and response variables
    y = final.loc[idx, 'Total_Revenue'] 
    x = final.loc[idx, X_cols]

    #add constant to predictor variables
    x = sm.add_constant(x)

    #fit linear regression model
    model1 = sm.OLS(y, x).fit()

    #view model summary
    print(model1.summary())

1993
1995
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
